In [30]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [31]:
# Dados
Tarefas = [i for i in range(6)]
proc_tempo = {0: 0, 1: 64, 2: 53, 3: 63, 4: 99, 5: 189}
entrega = {0: 0, 1: 100, 2: 70, 3: 150, 4: 601, 5: 118}
M = 10000

In [32]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.T = Set(initialize=Tarefas)
T = model.T

# Parâmetros
model.p = Param(T, initialize=proc_tempo, within=NonNegativeReals)
model.d = Param(T, initialize=entrega, within=NonNegativeReals)
p, d = model.p, model.d

# Variáveis de decisão
model.x = Var(T * T, within=Binary)  # Ordem de produção
model.C = Var(T, within=NonNegativeReals)  # Instante de conclusão
model.t = Var(T, within=NonNegativeReals)  # Tempo de atraso
model.y = Var(T, within=Binary)  # Tarefa atrasa ou não
model.t_max = Var(within=NonNegativeReals)  # Tempo máximo de atraso
x, C, t, y, t_max = model.x, model.C, model.t, model.y, model.t_max

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = t_max)

# Restrições
# Um sucessor por tarefa
model.r_sucessor = ConstraintList()
for i in T:
    model.r_sucessor.add(expr = sum(x[i, j] for j in T if j != i) == 1)
# Um antecessor por tarefa
model.r_antecessor = ConstraintList()
for j in T:
    model.r_antecessor.add(expr = sum(x[i, j] for i in T if i != j) == 1)
# Tempo de término
model.r_termino = ConstraintList()
for i in T:
    for j in T:
        if j != i and j > 0:
            model.r_termino.add(expr = C[j] >= C[i] + p[j] * x[i, j] + M * (x[i, j] - 1))
# Tempos de atraso positivos
model.r_atraso = ConstraintList()
for i in T:
    if i > 0:
        model.r_atraso.add(expr = t[i] >= C[i] - d[i])
# Binário de contagem de tarefas atrasadas
model.r_contagem_atraso = ConstraintList()
for i in T:
    model.r_contagem_atraso.add(expr = t[i] <= M * y[i])
# Tempo máximo
model.r_t_max = ConstraintList()
for i in T:
    model.r_t_max.add(expr = t_max >= t[i])

# model.pprint()

In [33]:
# Resolução
solver = SolverFactory('glpk')
resultado = solver.solve(model, tee=False)

In [34]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in T:
        for j in T:
            if i != j and value(x[i,j]) == 1:
                print('Tarefa {} precede a {}.'.format(i, j))    
    print()
    print('O atraso máximo é {:.0f} minutos'.format(value(model.obj)))
else:
    print(resultado.solver.termination_condition)

Tarefa 0 precede a 5.
Tarefa 1 precede a 3.
Tarefa 2 precede a 1.
Tarefa 3 precede a 4.
Tarefa 4 precede a 0.
Tarefa 5 precede a 2.

O atraso máximo é 219 minutos
